In [1]:
!nvidia-smi

Thu Jun 12 18:07:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.16                 Driver Version: 561.16         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   31C    P0              3W /   35W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import random
import shutil
from PIL import Image

base_dir = "dataset"
image_base = os.path.join(base_dir, "images")
label_base = os.path.join(base_dir, "labels")

In [ ]:
splits = ["train", "val"]
split_ratio = 0.8

In [ ]:
all_images = [f for f in os.listdir(image_base) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(all_images)

split_index = int(len(all_images) * split_ratio)
split_data = {
    "train": all_images[:split_index],
    "val": all_images[split_index:]
}

for split in splits:
    split_image_folder = os.path.join(image_base, split)
    os.makedirs(split_image_folder, exist_ok=True)
    for image_file in split_data[split]:
        src = os.path.join(image_base, image_file)
        dst = os.path.join(split_image_folder, image_file)
        shutil.move(src, dst)

In [ ]:
import cv2
import numpy as np

image_size = 400

base_path = f"{base_dir}/images"

for split in splits:
    folder_path = os.path.join(base_path, split)

    for image_file in os.listdir(folder_path):
        if not image_file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(folder_path, image_file)
        img = cv2.imread(image_path)

        if img is None:
            print(f"❌ Couldn't read: {image_path}")
            continue

        h, w = img.shape[:2]
        img_white = np.ones((image_size, image_size, 3), np.uint8) * 255

        aspect_ratio = h / w

        try:
            if aspect_ratio > 1:
                scale = image_size / h
                new_w = int(w * scale)
                resized_img = cv2.resize(img, (new_w, image_size))
                w_gap = (image_size - new_w) // 2
                img_white[:, w_gap:w_gap + new_w] = resized_img
            else:
                scale = image_size / w
                new_h = int(h * scale)
                resized_img = cv2.resize(img, (image_size, new_h))
                h_gap = (image_size - new_h) // 2
                img_white[h_gap:h_gap + new_h, :] = resized_img

            cv2.imwrite(image_path, img_white)

        except Exception as e:
            print(f"❌ Error processing {image_path}: {e}")


In [ ]:
class_names = [
    'A', 'B', 'C', '+', 'D', 'E', 'F', 'G', ',', 'H', 'I', '!', 'J',
    'K', 'L', 'M', 'N', 'O', '_', 'P', 'R', 'S', ';', 'T', 'U', '=',
    'V', 'Y', 'Z'
]
class_to_id = {name: idx for idx, name in enumerate(class_names)}

In [ ]:
for split in splits:
    image_folder = os.path.join(image_base, split)
    label_folder = os.path.join(label_base, split)
    os.makedirs(label_folder, exist_ok=True)

    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            try:
                image_path = os.path.join(image_folder, image_file)
                with Image.open(image_path) as img:
                    width, height = img.size

                class_name = image_file[0]
                if class_name not in class_to_id:
                    print(f"❌ Skipping unknown class: {class_name}")
                    continue

                class_id = class_to_id[class_name]
                x_center = 0.5
                y_center = 0.5
                norm_width = 1.0
                norm_height = 1.0

                label_line = f"{class_id} {x_center} {y_center} {norm_width} {norm_height}\n"
                label_filename = os.path.splitext(image_file)[0] + ".txt"
                label_path = os.path.join(label_folder, label_filename)

                with open(label_path, "w") as f:
                    f.write(label_line)

            except Exception as e:
                print(f"❌ Failed to process {image_file}: {e}")


In [4]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [8]:
model.train(
    data='dataset.yaml',
    epochs=20,
    batch=16,
    imgsz=400,
    project='tsl_project',
    name='tsl_yolo_train',
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.153  Python-3.12.8 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-13420H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=5, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.4, hsv_v=0.3, imgsz=400, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=tsl_yolo_train_2, nbs=64, nms=False, opset=None, optimize=False, 

train: Scanning C:\Users\narut\TSL-Alphabet-Detection\model\dataset\labels\train... 2377 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2377/2377 [00:02<00:00, 973.76it/s] 


train: New cache created: C:\Users\narut\TSL-Alphabet-Detection\model\dataset\labels\train.cache
val: Fast image access  (ping: 0.00.0 ms, read: 1117.6339.2 MB/s, size: 145.1 KB)


C:\Users\narut\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\narut\TSL-Alphabet-Detection\model\dataset\labels\val... 595 images, 0 backgrounds, 0 corrupt: 100%|██████████| 595/595 [00:00<00:00, 1002.06it/s]


val: New cache created: C:\Users\narut\TSL-Alphabet-Detection\model\dataset\labels\val.cache
Plotting labels to tsl_project\tsl_yolo_train_2\labels.jpg... 


...

       3/20         0G      1.201      4.785      1.656          2        416: 100%|██████████| 149/149 [05:27<00:00,  2.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:33<00:00,  1.77s/it]

                   all        595        595    0.00946       0.68     0.0324      0.018



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/20         0G      1.084      4.603      1.577          4        416:   7%|▋         | 11/149 [00:25<05:25,  2.36s/it]


KeyboardInterrupt: 